In [51]:
import QuantLib as ql

In [52]:
# market data setup
calendar = ql.TARGET()

settlementDate = ql.Date(18, 9, 2008)
# adjust date to make sure it's a business day
settlementDate = calendar.adjust(settlementDate)

fixingDays = 3
settlementDays = 3

todaysDate = calendar.advance(settlementDate, fixingDays * -1, ql.Days)
# set up evaluation date
ql.Settings.instance().evaluationDate = todaysDate

print(f'Today: {todaysDate.ISO()}')
print(f'Settlement date: {settlementDate.ISO()}')

Today: 2008-09-15
Settlement date: 2008-09-18


Building of the bonds discounting yield curve

*********************
***  RATE HELPERS ***
*********************/

RateHelpers are built from the above quotes together with
other instrument dependant infos.  Quotes are passed in
relinkable handles which could be relinked to some other data source later.


In [53]:
# common data
# zero coupon rate for the short end
zc3mQuote = 0.0096
zc6mQuote = 0.0145
zc1yQuote = 0.0194

zc3mRate = ql.SimpleQuote(zc3mQuote)
zc6mRate = ql.SimpleQuote(zc6mQuote)
zc1yRate = ql.SimpleQuote(zc1yQuote)

zcBondsDayCounter = ql.Actual365Fixed()

# define rate helper for short end
zc3m = ql.DepositRateHelper(ql.QuoteHandle(zc3mRate), ql.Period(3, ql.Months),
                            fixingDays, calendar, ql.ModifiedFollowing, True, zcBondsDayCounter)
zc6m = ql.DepositRateHelper(ql.QuoteHandle(zc6mRate), ql.Period(6, ql.Months),
                            fixingDays, calendar, ql.ModifiedFollowing, True, zcBondsDayCounter)
zc1y = ql.DepositRateHelper(ql.QuoteHandle(zc1yRate), ql.Period(1, ql.Years),
                            fixingDays, calendar, ql.ModifiedFollowing, True, zcBondsDayCounter)

# setup bonds
redemption = 100.0
numberOfBonds = 5
issueDates = []
issueDates.append(ql.Date(15, 3, 2005))
issueDates.append(ql.Date(15, 6, 2005))
issueDates.append(ql.Date(30, 6, 2006))
issueDates.append(ql.Date(15, 11, 2002))
issueDates.append(ql.Date(15, 5, 1987))

maturities = []
maturities.append(ql.Date(31, 8, 2010))
maturities.append(ql.Date(31, 8, 2011))
maturities.append(ql.Date(31, 8, 2013))
maturities.append(ql.Date(15, 8, 2018))
maturities.append(ql.Date(15, 5, 2038))

couponRate = [0.02375, 0.04625, 0.03125, 0.04, 0.045]

marketQuotes = [100.390625, 106.21875, 100.59375, 101.6875, 102.140625]

quotes = []
for i in range(numberOfBonds):
    cp = ql.SimpleQuote(marketQuotes[i])
    quotes.append(cp)

quoteHandles = []
for i in range(numberOfBonds):
    rlq = ql.QuoteHandle(quotes[i])
    quoteHandles.append(rlq)

# define the rate helper for bonds - FixedRateBondHelper
bondsHelpers=[]
for i in range(numberOfBonds):
    schedule = ql.MakeSchedule(issueDates[i], maturities[i], frequency=ql.Semiannual, 
                               calendar=ql.TARGET(), convention=ql.Following, 
                               terminalDateConvention=ql.Following, backwards=True)
    cpn=[]
    cpn.append(couponRate[i])
    dc = ql.ActualActual(ql.ActualActual.Bond)
    bondHelper = ql.FixedRateBondHelper(quoteHandles[i],
                                        settlementDays,
                                        100.0,
                                        schedule,
                                        cpn,
                                        dc,
                                        ql.Unadjusted,
                                        redemption,
                                        issueDates[i])
    bondsHelpers.append(bondHelper)


*********************
**  CURVE BUILDING **
*********************

In [54]:
# speficy the day counter for yield term structure
# any DayCounter would be fine.
termStructureDayCounter = ql.ActualActual(ql.ActualActual.ISDA)

tolerance = 1.0e-15

# A depo-bond curve
bondInstruments = []

# Adding the ZC bonds to the curve for the short end
bondInstruments.append(zc3m)
bondInstruments.append(zc6m)
bondInstruments.append(zc1y)

# adding the Fixed rate bonds to the curve for the long end
for i in range(numberOfBonds):
    bondInstruments.append(bondsHelpers[i])

bondDiscountingTermStructure = ql.PiecewiseLogLinearDiscount(settlementDate,
                                                             bondInstruments, 
                                                             termStructureDayCounter)



*********************
FIXED RATE BONDS TO BE PRICED
**********************

In [55]:
# setup bond face amount
faceAmount = 100.0

# define pricing engine
discountingHandler = ql.RelinkableYieldTermStructureHandle(bondDiscountingTermStructure)
bondDiscountEngine = ql.DiscountingBondEngine(discountingHandler)

# setup Fixed Rate Bond
fixedBondSchedule = ql.MakeSchedule(ql.Date(15,5,2007),
                                    ql.Date(15,5,2017),
                                    frequency=ql.Semiannual, 
                                    calendar=ql.UnitedStates(ql.UnitedStates.GovernmentBond), 
                                    convention=ql.Following, 
                                    terminalDateConvention=ql.Following, 
                                    forwards=True)
cpn=[0.045]
dc = ql.ActualActual(ql.ActualActual.Bond)

fixedRateBond = ql.FixedRateBond(
    settlementDays,
    faceAmount,
    fixedBondSchedule,
    cpn,
    dc,
    ql.ModifiedFollowing,
    100.0,
    ql.Date(15,5,2007))

fixedRateBond.setPricingEngine(bondDiscountEngine)

print(f'Fixed Rate Bond NPV is : {fixedRateBond.NPV()}')
print(f'Fixed Rate Bond Clean Price is : {fixedRateBond.cleanPrice()}')
print(f'Fixed Rate Bond Dirty Price is : {fixedRateBond.dirtyPrice()}')
print(f'Fixed Rate Bond Accrued coupon is : {fixedRateBond.accruedAmount()}')
print(f'Fixed Rate Bond Previous coupon is : {fixedRateBond.previousCouponRate()}')
print(f'Fixed Rate Bond Next coupon is : {fixedRateBond.nextCouponRate()}')
print(f'Fixed Rate Bond Yield is {fixedRateBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual)}')


cfs = fixedRateBond.cashflows()

Fixed Rate Bond NPV is : 107.67108123119635
Fixed Rate Bond Clean Price is : 106.14688768280926
Fixed Rate Bond Dirty Price is : 107.67108123119635
Fixed Rate Bond Accrued coupon is : 1.5241935483870872
Fixed Rate Bond Previous coupon is : 0.045
Fixed Rate Bond Next coupon is : 0.045
Fixed Rate Bond Yield is 0.0364718979358673


In [56]:
import pandas as pd

cashflows = pd.DataFrame({
    'rate': cf.date().ISO(),
    'amount': cf.amount()
} for cf in fixedRateBond.cashflows())

display(cashflows)

,rate,amount
0,2007-11-15,2.25
1,2008-05-15,2.25
2,2008-11-17,2.25
3,2009-05-15,2.25
4,2009-11-16,2.25
5,2010-05-17,2.25
6,2010-11-15,2.25
7,2011-05-16,2.25
8,2011-11-15,2.25
9,2012-05-15,2.25


In [57]:
# price a zero coupon bond
zeroCouponBond = ql.ZeroCouponBond(
    settlementDays,
    ql.UnitedStates(ql.UnitedStates.GovernmentBond),
    faceAmount,
    ql.Date(15,8,2013),
    ql.Following,
    116.92,
    ql.Date(15,8,2003))

zeroCouponBond.setPricingEngine(bondDiscountEngine)

print(f'Zero Coupon Bond NPV is : {zeroCouponBond.NPV()}')
print(f'Zero Coupon Bond Clean Price is : {zeroCouponBond.cleanPrice()}')
print(f'Zero Coupon Bond Dirty Price is : {zeroCouponBond.dirtyPrice()}')
print(f'Zero Coupon Bond Accrued coupon is : {zeroCouponBond.accruedAmount()}')
print(f'Zero Coupon Bond Yield is {zeroCouponBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual)}')


Zero Coupon Bond NPV is : 100.93296083794954
Zero Coupon Bond Clean Price is : 100.93296083794954
Zero Coupon Bond Dirty Price is : 100.93296083794954
Zero Coupon Bond Accrued coupon is : 0.0
Zero Coupon Bond Yield is 0.02997852950096131


*********************
FLOATING RATE BONDS TO BE PRICED
**********************

In [60]:
# Building of the libor forecasting curve
# Money Market rate
d1wQuote = 0.0382
d1mQuote = 0.0372
d3mQuote = 0.0363
d6mQuote = 0.0353
d9mQuote = 0.0348
d1yQuote = 0.0345
# swap rate
s2yQuote = 0.037125
s3yQuote = 0.0398
s5yQuote = 0.0443
s10yQuote = 0.05165
s15yQuote = 0.055175

# define Quotes
# SimpleQuotes stores a value which can be manually changed;
# deposit rate (money market rate)
d1wRate = ql.QuoteHandle(ql.SimpleQuote(d1wQuote))
d1mRate = ql.QuoteHandle(ql.SimpleQuote(d1mQuote))
d3mRate = ql.QuoteHandle(ql.SimpleQuote(d3mQuote))
d6mRate = ql.QuoteHandle(ql.SimpleQuote(d6mQuote))
d9mRate = ql.QuoteHandle(ql.SimpleQuote(d9mQuote))
d1yRate = ql.QuoteHandle(ql.SimpleQuote(d1yQuote))
# swap rates
s2yRate = ql.QuoteHandle(ql.SimpleQuote(s2yQuote))
s3yRate = ql.QuoteHandle(ql.SimpleQuote(s3yQuote))
s5yRate = ql.QuoteHandle(ql.SimpleQuote(s5yQuote))
s10yRate = ql.QuoteHandle(ql.SimpleQuote(s10yQuote))
s15yRate = ql.QuoteHandle(ql.SimpleQuote(s15yQuote))

# setup Rate Helpers

fixingDays = 2
endOfMonth = False
# deposits
depositDayCounter = ql.Actual360()
# there's error when we using Qutoe object for creating instance
d1w = ql.DepositRateHelper(d1wRate, ql.Period('1W'), fixingDays, calendar, ql.ModifiedFollowing, endOfMonth,depositDayCounter)
d1m = ql.DepositRateHelper(d1mRate, ql.Period('1M'), fixingDays, calendar, ql.ModifiedFollowing, endOfMonth,depositDayCounter)
d3m = ql.DepositRateHelper(d3mRate, ql.Period('3M'), fixingDays, calendar, ql.ModifiedFollowing, endOfMonth,depositDayCounter)
d6m = ql.DepositRateHelper(d6mRate, ql.Period('6M'), fixingDays, calendar, ql.ModifiedFollowing, endOfMonth,depositDayCounter)
d9m = ql.DepositRateHelper(d9mRate, ql.Period('9M'), fixingDays, calendar, ql.ModifiedFollowing, endOfMonth,depositDayCounter)
d1y = ql.DepositRateHelper(d1yRate, ql.Period('1Y'), fixingDays, calendar, ql.ModifiedFollowing, endOfMonth,depositDayCounter)

# swap instruments
swFixedLegFrequency = ql.Annual
swFixedLegConvention = ql.Unadjusted
swFixedLegDayCounter = ql.Thirty360(ql.Thirty360.European)

swFloatingLegIndex = ql.Euribor6M()

s2y = ql.SwapRateHelper(s2yRate, ql.Period('2Y'), calendar, swFixedLegFrequency, swFixedLegConvention, swFixedLegDayCounter, swFloatingLegIndex)
s3y = ql.SwapRateHelper(s3yRate, ql.Period('3Y'), calendar, swFixedLegFrequency, swFixedLegConvention, swFixedLegDayCounter, swFloatingLegIndex)
s5y = ql.SwapRateHelper(s5yRate, ql.Period('5Y'), calendar, swFixedLegFrequency, swFixedLegConvention, swFixedLegDayCounter, swFloatingLegIndex)
s10y = ql.SwapRateHelper(s10yRate, ql.Period('10Y'), calendar, swFixedLegFrequency, swFixedLegConvention, swFixedLegDayCounter, swFloatingLegIndex)
s15y = ql.SwapRateHelper(s15yRate, ql.Period('15Y'), calendar, swFixedLegFrequency, swFixedLegConvention, swFixedLegDayCounter, swFloatingLegIndex)

# curve building
termStructureDayCounter = ql.ActualActual(ql.ActualActual.ISDA) 
tolerance = 1.0e-15

#build curve
helpers = ql.RateHelperVector()
helpers.append(d1w)
helpers.append(d1m)
helpers.append(d3m)
helpers.append(d6m)
helpers.append(d9m)
helpers.append(d1y)
helpers.append(s2y)
helpers.append(s3y)
helpers.append(s5y)
helpers.append(s10y)
helpers.append(s15y)

depoSwapTermStructure = ql.PiecewiseLogLinearDiscount(todaysDate, helpers, 
                                                      termStructureDayCounter)

forecastingHandler = ql.RelinkableYieldTermStructureHandle(depoSwapTermStructure)

# setup floating bond
floatingIndex = ql.Euribor6M(forecastingHandler)
floatingIndex.addFixing(ql.Date(18,10,2007), 0.027)
floatingIndex.addFixing(ql.Date(17,4,2008), 0.0278625)
floatingSchedule = ql.MakeSchedule(ql.Date(21,10,2005),
                                    ql.Date(21,10,2010),
                                    frequency=ql.Semiannual, 
                                    calendar=ql.UnitedStates(ql.UnitedStates.NYSE), 
                                    convention=ql.Unadjusted, 
                                    terminalDateConvention=ql.Unadjusted, 
                                    backwards=True)
floatingBond = ql.FloatingRateBond(
    settlementDays,
    faceAmount,
    floatingSchedule,
    floatingIndex,
    ql.Actual360(),
    ql.ModifiedFollowing,
    2,
    [1.0],
    [0.001],
    [],
    [],
    False,
    100.0,
    ql.Date(21,10,2005))

floatingBond.setPricingEngine(bondDiscountEngine)

print(f'Floating Rate Bond NPV is : {floatingBond.NPV()}')
print(f'Floating Rate Bond Clean Price is : {floatingBond.cleanPrice()}')
print(f'Floating Rate Bond Dirty Price is : {floatingBond.dirtyPrice()}')
print(f'Floating Rate Bond Accrued coupon is : {floatingBond.accruedAmount()}')
print(f'Floating Rate Bond Previous coupon is : {floatingBond.previousCouponRate()}')
print(f'Floating Rate Bond Next coupon is : {floatingBond.nextCouponRate()}')
print(f'Floating Rate Bond Yield is {floatingBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual)}')



Floating Rate Bond NPV is : 104.35378848120487
Floating Rate Bond Clean Price is : 103.1511843145382
Floating Rate Bond Dirty Price is : 104.35378848120486
Floating Rate Bond Accrued coupon is : 1.2026041666666667
Floating Rate Bond Previous coupon is : 0.028
Floating Rate Bond Next coupon is : 0.0288625
Floating Rate Bond Yield is 0.022040569639205934


In [71]:
# display cash flows table by pandas
import pandas as pd

floatingRates=[]
floatingAmounts=[]
accStartDates=[]
accEndDates=[]
for cf in map(ql.as_floating_rate_coupon, floatingBond.cashflows()):
    try:
        floatingRates.append(cf.rate())
        floatingAmounts.append(cf.amount())
        accStartDates.append(cf.accrualStartDate().ISO())
        accEndDates.append(cf.accrualEndDate().ISO())
    except:
        pass

cashflows={
    'StartDate': accStartDates,
    'EndDate': accEndDates,
    'Rates': floatingRates,
    'Amounts': floatingAmounts
}
df = pd.DataFrame(cashflows)
display(df)

,StartDate,EndDate,Rates,Amounts
0,2007-10-21,2008-04-21,0.028000,1.423333
1,2008-04-21,2008-10-21,0.028862,1.467177
2,2008-10-21,2009-04-21,0.035554,1.797431
3,2009-04-21,2009-10-21,0.035064,1.782439
4,2009-10-21,2010-04-21,0.039441,1.993940
5,2010-04-21,2010-10-21,0.040500,2.058725
